In [ ]:
pip install -Uqq fastbook


In [ ]:
from fastbook import *
from fastai.vision.all import *

In [ ]:
dest_path = Path('/content/MLdata')
path = untar_data(URLs.MNIST_SAMPLE, dest_path)
path

Path('/root/.fastai/data/mnist_sample')

In [ ]:
path.ls()


(#3) [Path('train'),Path('valid'),Path('labels.csv')]

In [ ]:
Path.BASE_PATH = path

In [ ]:
three = (path/'train/3').ls().sorted()
seven = (path/'train/7').ls().sorted()

In [ ]:
tensor3=[tensor(Image.open(f))for f in three]
tensor7=[tensor(Image.open(f))for f in seven]

In [ ]:
stacked3 = torch.stack(tensor3).float()/255
stacked7 = torch.stack(tensor7).float()/255

In [ ]:
three = (path/'valid/3').ls().sorted()
seven = (path/'valid/7').ls().sorted()

In [ ]:
valid_tensor3=[tensor(Image.open(f))for f in three]
valid_tensor7=[tensor(Image.open(f))for f in seven]


In [ ]:
valid_stacked3 = torch.stack(valid_tensor3).float()/255
valid_stacked7 = torch.stack(valid_tensor7).float()/255

In [ ]:
mean3 = stacked3.mean(0)
mean7 = stacked7.mean(0)

In [ ]:
def img_distance(input, mean_tensor):
  return (input - mean_tensor).abs().mean((-1,-2))
  

In [ ]:
def is_3(x):
  return (img_distance(x, mean3) < img_distance(x, mean7)).float()

In [ ]:
def answer(a,b):
  return (a-b).abs().mean()

In [ ]:
trainX = torch.cat([stacked3, stacked7]).view(-1, 28*28)
trainY = tensor([1]*len(stacked3) + [0]*len(stacked7)).unsqueeze(-1)

In [ ]:
valid_trainX = torch.cat([valid_stacked3, valid_stacked7]).view(-1, 28*28)
valid_trainY = tensor([1]*len(valid_stacked3) + [0]*len(valid_stacked7)).unsqueeze(-1)

In [ ]:
dset = list(zip(trainX, trainY))
valid_dset = list(zip(valid_trainX, valid_trainY))

In [ ]:
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256)

In [73]:
def init_params(size, std=1.0):
  return ((torch.randn(size)*std)requires_grad_())

In [76]:
weights = init_params((28*28,1))
bias = init_params(1)
bias = Variable(bias , requires_grad = True)
weights = Variable(weights , requires_grad = True)
params = weights, bias


AttributeError: ignored

In [ ]:
def mnist_loss(preds, target):
  preds = preds.sigmoid()
  preds = (preds > 0.5).float()
  # error = []
  # for i in range(len(target)):
  #   if target[i]==1:
  #     error.append(1-preds[i])
  #   else:
  #     error.append(preds[i])
  #   errors= tensor(error).mean()
  #   return errors
  return torch.where(target==1,1-preds, preds).mean()

In [ ]:
def linear1(xb):
  return xb@weights + bias

In [ ]:
from torch.autograd import Variable
def calc_grad(xb, yb, model):
  preds = model(xb)
  loss = mnist_loss(preds, yb)
  loss = Variable(loss, requires_grad = True)
  loss.backward()

In [ ]:
calc_grad(list(dl)[20][0][200],list(dl)[20][1][200], linear1)

None


In [ ]:
def train_epoch(model, lr, params):
  for xb,yb in dl:
    calc_grad(xb, yb, model)
    for p in params:
      p = Variable(p, requires_grad=True)
      p.data -= p.grad*lr
      p.grad.zero_()

In [ ]:
def batch_accuracy(xb_pred, yb):
  preds = xb_pred.sigmoid()
  correct = (preds > 0.5) == yb
  return correct.float().mean()
  

In [ ]:
def valid_epoch(model):
  accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
  return round(torch.stack(accs).mean().item(), 4)

In [74]:
for ep in range(10):
  train_epoch(linear1,0.1,params)
  print(valid_epoch(linear1))


TypeError: ignored